In [5]:
import pandas as pd
import random
import math
import datetime
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans, spectral_clustering, SpectralClustering
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from numpy.random import normal
# from graspy.simulations import sbm
import numpy as np
from scipy.sparse import csr_matrix
from numpy.linalg import norm

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

In [4]:
mnist = pd.read_csv('mnist_train.csv')

In [714]:
def get_B_and_weight_vec(matrix,threshold):
    N = matrix.shape[0]
#     clustering = SpectralClustering(n_clusters=2, assign_labels="discretize", random_state=0).fit(points)
    A = np.copy(matrix)

    row = []
    col = []
    data = []
    weight_vec = []
    cnt = 0
    for i in range(N):
        for j in range(N):
            if j <= i:
                continue
            if A[i, j] < threshold: 
                A[i, j] = 0
                A[j, i] = 0
                continue
            row.append(cnt)
            col.append(i)
            data.append(1)

            row.append(cnt)
            col.append(j)
            data.append(-1)
            cnt += 1
            weight_vec.append(A[i, j])

    B = csr_matrix((data, (row, col)), shape=(cnt, N))
    weight_vec = np.array(weight_vec)
    return A, B, weight_vec

In [715]:

def algorithm(B, weight_vec, seeds, K=15000, alpha=0.01, lambda_nLasso=None, check_s=False):
    E, N = B.shape
#     weight_vec = np.ones(E)

    Gamma_vec = np.array(1./(np.sum(abs(B), 0)))[0]  # \in [0, 1]
    Gamma = np.diag(Gamma_vec)

    Sigma = 0.5
    samplingset = seeds

    seednodesindicator= np.zeros(N)
    seednodesindicator[samplingset] = 1
    
    
    noseednodeindicator = np.ones(N)
    noseednodeindicator[samplingset] = 0
    
    
    if lambda_nLasso == None:
        lambda_nLasso = 2 / math.sqrt(np.sum(weight_vec))
    
    if check_s:
        s = 0.0
        for item in range(len(weight_vec)):
            x = B[item].toarray()[0]
            i = np.where(x == -1)[0][0]
            j = np.where(x == 1)[0][0]
            if i < N1 <= j:
                s += weight_vec[item]
            elif i >= N1 > j:
                s += weight_vec[item]

        if lambda_nLasso * s >= alpha * N2 / 2:
            print ('eq(24)', lambda_nLasso * s, alpha * N2 / 2)
    
    fac_alpha = 1./(Gamma_vec*alpha+1)  # \in [0, 1]

    hatx = np.zeros(N)
    newx = np.zeros(N)
    prevx = np.zeros(N)
    haty = np.array([x/(E-1) for x in range(0, E)])
    history = []
    for iterk in range(K):
        # if 0 < np.max(abs(newx - prevx)) < 1e-4:
        #     print(iterk)
        #     break
        tildex = 2 * hatx - prevx
        newy = haty + Sigma * B.dot(tildex)  # chould be negative
        haty = newy / np.maximum(abs(newy) / (lambda_nLasso * weight_vec), np.ones(E))  # could be negative

        newx = hatx - Gamma_vec * B.T.dot(haty)  # could  be negative
        newx[samplingset] = (newx[samplingset] + Gamma_vec[samplingset]) / (1 + Gamma_vec[samplingset])

        newx = seednodesindicator * newx + noseednodeindicator * (newx * fac_alpha)
        prevx = np.copy(hatx)
        hatx = newx  # could be negative
        history.append(newx)
    
    history = np.array(history)

    return history
    
    

# image custering on MNIST dataset

## 2 clusters


In [747]:
def get_digits_2clusters():
    '''
    Randomly select 200 images of digit "1" and "3"
    '''
    mnist_3 = mnist[mnist['label']==3].iloc[:1000,:]

    mnist_1 = mnist[mnist['label']==1].iloc[:1000,:]

    digits=mnist_3.append(mnist_1)
    digits = digits.sample(frac=0.1).reset_index(drop=True)
    return digits


In [138]:
def run_digit(affinity_matrix, threshold,seeds, K=1000, alpha=0.01, lambda_nLasso=0.1):

    A,B, weight_vec = get_B_and_weight_vec(affinity_matrix,threshold)
    
    start = datetime.datetime.now()
    history = algorithm(B, weight_vec,seeds, K=K, alpha=alpha, lambda_nLasso=lambda_nLasso)
    print('our method time: ', datetime.datetime.now() - start)
    
    start = datetime.datetime.now()
    spectral_labels = spectral_clustering(affinity_matrix,n_clusters=2)
    print ('spectral clustering time: ', datetime.datetime.now() - start)
    return history,spectral_labels

In [748]:
def get_weig_mat_2clusters(digits):
    '''
    Generate adjacency matrix  based on distance measure in high dimentional space
    '''
    weig_mat_2clusters = np.zeros((200,200))   
    for i in range(200):
        for j in range(200):
            if i == j:
                continue
            weig_mat_2clusters[i][j]=np.exp(-norm(digits.iloc[i,1:]-digits.iloc[j,1:])/1000)
    return weig_mat_2clusters


In [736]:
def run_digit(matrix, threshold,K=1000, seeds=seeds, alpha=0.2, lambda_nLasso=0.1):
    A,B, weight_vec = get_B_and_weight_vec(matrix,threshold)
    
    start = datetime.datetime.now()
    history = algorithm(B, weight_vec,seeds = seeds, K=K, alpha=alpha, lambda_nLasso=lambda_nLasso)
    print('our method time: ', datetime.datetime.now() - start)
    
    start = datetime.datetime.now()
    spectral_labels = spectral_clustering(matrix,n_clusters=2)
#     spectral_labels = SpectralClustering(2).fit(digits.iloc[:,1:]).labels_
    print ('spectral clustering time: ', datetime.datetime.now() - start)
    
    return history,spectral_labels

In [755]:
# Repeatedly run the experiment
our_accus_2clusters = []
spec_accus_2clusters = []
for i in range(20):
    digits = get_digits_2clusters()
    true_labels = np.array([0 if i==3 else 1 for i in digits.label])
    weig_mat_2clusters = get_weig_mat_2clusters(digits)
    num_seeds = 0.2*100
    seeds = np.random.choice(np.argwhere(digits.label.values==1).squeeze(),int(num_seeds))
    history,spectral_labels = run_digit(weig_mat_2clusters,threshold=0.25,K=300, seeds=seeds, \
                                        alpha=0.01, lambda_nLasso=0.1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(np.nan_to_num(history[-1].reshape(-1,1)))
    if sum(kmeans.labels_ == true_labels) >= 100:
        our_accu = sum(kmeans.labels_ == true_labels)/len(true_labels)
    else:
        our_accu =(len(true_labels)-sum(kmeans.labels_ == true_labels))/len(true_labels)
    
    our_accus_2clusters.append(our_accu)
    if sum(spectral_labels == true_labels) >= 100:
        spec_accu = sum(spectral_labels== true_labels)/len(true_labels)
    else:
        spec_accu = (len(true_labels)-sum(spectral_labels == true_labels))/len(true_labels)
    spec_accus_2clusters.append(spec_accu)

our method time:  0:00:00.039293
spectral clustering time:  0:00:00.023383
our method time:  0:00:00.042036
spectral clustering time:  0:00:00.019869
our method time:  0:00:00.054367
spectral clustering time:  0:00:00.032394
our method time:  0:00:00.046315
spectral clustering time:  0:00:00.033510
our method time:  0:00:00.039756
spectral clustering time:  0:00:00.019892
our method time:  0:00:00.040082
spectral clustering time:  0:00:00.021559
our method time:  0:00:00.048948
spectral clustering time:  0:00:00.025837
our method time:  0:00:00.043275
spectral clustering time:  0:00:00.024941
our method time:  0:00:00.044286
spectral clustering time:  0:00:00.022470
our method time:  0:00:00.041615
spectral clustering time:  0:00:00.021209
our method time:  0:00:00.044696
spectral clustering time:  0:00:00.030415
our method time:  0:00:00.039906
spectral clustering time:  0:00:00.020732
our method time:  0:00:00.038186
spectral clustering time:  0:00:00.020011
our method time:  0:00:00

In [757]:
np.mean(our_accus_2clusters), np.mean(spec_accus_2clusters),np.std(our_accus_2clusters),np.std(spec_accus_2clusters)

(0.9842500000000001,
 0.9279999999999999,
 0.008104782538723671,
 0.06038211655780212)

### 3 clusters

In [493]:
import random
import math
import itertools
import numpy as np
from sklearn.cluster import spectral_clustering, SpectralClustering
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
import datetime
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture

n_samples = 6000

def accuracy(labels, true_labels):
    total_common = 0
    cluster_names = set(true_labels)
    permutations = list(itertools.permutations(cluster_names))
    for permutation in permutations:
        max_common = 0
        for i, cluster_name in enumerate(cluster_names):
            cluster_nodes = np.where(labels == cluster_name)[0]
            cluster_name1 = permutation[i]
            true_nodes = np.where(true_labels == cluster_name1)[0]

            common = len(set(true_nodes) - (set(true_nodes) - set(cluster_nodes)))
            max_common += common

        total_common = max(total_common, max_common)

    return total_common / len(true_labels)

def get_B_and_weight_vec_digits(matrix):
    A = np.copy(matrix)
    N = matrix.shape[0]

    row = []
    col = []
    data = []
    weight_vec = []
    cnt = 0
    for i in range(N):
        for j in range(N):
            if j <= i:
                continue
            if A[i, j] < 0.15:
                A[i, j] = 0
                A[j, i] = 0
                continue
            row.append(cnt)
            col.append(i)
            data.append(1)

            row.append(cnt)
            col.append(j)
            data.append(-1)
            cnt += 1
            weight_vec.append(A[i, j])

    B = csr_matrix((data, (row, col)), shape=(cnt, N))
    weight_vec = np.array(weight_vec)
    return B, weight_vec


def run_multi_digits(matrix, true_labels, K, alpha, lambda_nLasso,n_clusters, M=0.2):
    B, weight_vec = get_B_and_weight_vec_digits(matrix)
    E,N=B.shape[0],B.shape[1]

    Gamma_vec = np.array(1. / (np.sum(abs(B), 0)))[0]  # \in [0, 1]
    Gamma = np.diag(Gamma_vec)

    Sigma = 0.5

    fac_alpha = 1. / (Gamma_vec * alpha + 1)  # \in [0, 1]
    lambda_weight = lambda_nLasso * weight_vec

    our_labels = np.full(N, n_clusters-1)
    our_time = datetime.datetime.now() - datetime.datetime.now()
    for clust_num in range(n_clusters-1):

        samplingset = random.choices(np.where(true_labels==clust_num)[0], k=int(M * len(np.where(true_labels==clust_num)[0])))
        seednodesindicator = np.zeros(N)
        seednodesindicator[samplingset] = 1
        noseednodeindicator = np.ones(N)
        noseednodeindicator[samplingset] = 0


        hatx = np.zeros(N)
        newx = np.zeros(N)
        prevx = np.zeros(N)
        haty = np.array([x / (E - 1) for x in range(0, E)])
        gamma_plus = 1 + Gamma_vec[samplingset]
        start = datetime.datetime.now()
        for iterk in range(K):
            tildex = 2 * hatx - prevx
            newy = haty + Sigma * B.dot(tildex)  # chould be negative
            res = abs(newy) / lambda_weight
            res[res < 1] = 1
            haty = newy / res

            newx = hatx - Gamma_vec * B.T.dot(haty)  # could  be negative

            newx[samplingset] = (newx[samplingset] + Gamma_vec[samplingset]) / gamma_plus

            newx = seednodesindicator * newx + noseednodeindicator * (newx * fac_alpha)
            prevx = np.copy(hatx)
            hatx = newx  # could be negative
        our_time += datetime.datetime.now() - start
        X = newx
        X = np.nan_to_num(X, 0)
        kmeans = KMeans(n_clusters=2, random_state=0).fit(X.reshape(len(X), 1))
        matched_label = kmeans.labels_[samplingset][0]
        our_labels[np.where(kmeans.labels_ == matched_label)[0]] = clust_num
    
    print ('our time is:', our_time)
    
    start = datetime.datetime.now()
    labels = spectral_clustering(matrix, n_clusters=3,random_state=1)
#     labels = SpectralClustering(3).fit(X056).labels_
    print ('spectral clustering time is:', datetime.datetime.now() - start)
    our_accuracy = accuracy(our_labels, true_labels)
    spec_accuracy = accuracy(labels, true_labels)
    
    return our_accuracy, spec_accuracy



In [551]:
def get_digits_3clusters():
    '''
    Randomly select 300 images of digit "0","5" and "6"
    '''
    mnist_5 = mnist[mnist['label']==5].iloc[:1000,:].sample(frac=0.1).reset_index(drop=True)
    mnist_0 = mnist[mnist['label']==0].iloc[:1000,:].sample(frac=0.1).reset_index(drop=True)
    mnist_6 = mnist[mnist['label']==6].iloc[:1000,:].sample(frac=0.1).reset_index(drop=True)
    digits_3clusters = mnist_0.append(mnist_5,ignore_index=True).append(mnist_6,ignore_index=True)
    return digits_3clusters


In [552]:

def get_weig_mat_3clusters(digits):
    '''
    Generate adjacency matrix based on distance measure in high dimentional space
    '''
    weig_mat_3clusters = np.zeros((300,300))    #adjacency matrix based on neighbor probabilty.
    for i in range(300):
        for j in range(300):
            if i == j:
                continue
            weig_mat_3clusters[i][j]=np.exp(-norm(digits.iloc[i,1:]-digits.iloc[j,1:])/1000)
    return weig_mat_3clusters


In [688]:
# Repeatedly run the experiment
true_labels = np.array([0 for i in range(100)] + [1 for i in range(100)] + [2 for i in range(100)])
our_accus = []
spec_accus = []
for i in range(10):
    digits_3clusters = get_digits_3clusters()
    weig_mat_3clusters = get_weig_mat_3clusters(digits_3clusters)
    
    our_accu, spec_accu = run_multi_digits(matrix=weig_mat_3clusters,true_labels=true_labels, K=300,\
                 alpha=0.01, lambda_nLasso=0.2,n_clusters=3)
    our_accus.append(our_accu)
    spec_accus.append(spec_accu)


our time is: 0:00:00.095289
spectral clustering time is: 0:00:00.051400
our time is: 0:00:00.094048
spectral clustering time is: 0:00:00.043544
our time is: 0:00:00.091935
spectral clustering time is: 0:00:00.040619
our time is: 0:00:00.093330
spectral clustering time is: 0:00:00.039564
our time is: 0:00:00.094307
spectral clustering time is: 0:00:00.042665
our time is: 0:00:00.094526
spectral clustering time is: 0:00:00.034100
our time is: 0:00:00.095031
spectral clustering time is: 0:00:00.039788
our time is: 0:00:00.091518
spectral clustering time is: 0:00:00.045876
our time is: 0:00:00.102445
spectral clustering time is: 0:00:00.046759
our time is: 0:00:00.094498
spectral clustering time is: 0:00:00.042947


In [693]:
our_accus

[0.8866666666666667,
 0.8566666666666667,
 0.91,
 0.5833333333333334,
 0.9466666666666667,
 0.8433333333333334,
 0.9466666666666667,
 0.92,
 0.8933333333333333,
 0.89]

In [694]:
np.mean(our_accus),np.mean(spec_accus),np.std(our_accus),np.std(spec_accus)

(0.8676666666666668, 0.752, 0.10004498987955368, 0.07578038444522876)